In [23]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)


In [24]:
input_query=inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [25]:
input_1=inputs[0]

In [26]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [27]:
i=0 

res=torch.dot(inputs[i], input_query)
res

tensor(0.9544)

In [28]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [29]:
attn_weights_2_tmp=attn_scores_2/attn_scores_2.sum()


In [30]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_naive(attn_scores_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [31]:
attn_scores_2_nm = torch.softmax(attn_scores_2, dim=0)


In [32]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_scores_2_nm[i]*x_i

print(context_vec_2)



tensor([0.4419, 0.6515, 0.5683])


In [33]:
attn_scores=torch.empty(6,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i, x_j) 

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [34]:
attn_scores= inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [35]:
attn_weights = torch.softmax(attn_scores, dim=1)

In [36]:
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [37]:
all_context_vecs=attn_weights @ inputs 
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [46]:
x_2 = inputs[1]
d_in=inputs.shape[1]
d_out=2



In [51]:
torch.manual_seed(123)

W_query=torch.nn.Parameter(torch.rand(d_in, d_out))
W_key=torch.nn.Parameter(torch.rand(d_in, d_out))
W_value=torch.nn.Parameter(torch.rand(d_in, d_out))


In [49]:
query_2 = x_2 @ W_query 

query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [53]:
keys=inputs @ W_key 
value=inputs@ W_value

keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)

In [58]:
keys_2=keys[1]
attn_score_22=torch.dot(query_2, keys_2)

attn_score_2 = query_2 @ keys.T 
attn_score_2 

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [63]:
d_k=keys.shape[1]

attn_weights=torch.softmax(attn_score_2/d_k**0.5,dim=-1)
attn_weights.sum()



tensor(1., grad_fn=<SumBackward0>)

In [66]:
context_vec_2=attn_weights @ value

context_vec_2

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)